# INTEGRATED USE OF MULTISOURCE REMOTE SENSING DATA FOR NATIONAL SCALE AGRICULTURAL  DROUGHT MONITORING IN KENYA
# ADM-Kenya, 2024
# High-Resolution Evaportranspiration Estimation

Based on the review of state-of-the-art methods to estimate ET using Sentinel-3 the following processing chain was developed for Land Surface Temperature (LST) downscaling and Evapotranspiration (ET) estimation.

# Tools and Libraries

We will use the following libraries:

    NumPy: A fundamental package for scientific computing in Python.
    Rasterio: A library for reading and writing geospatial raster datasets.
    GDAL: A library for working with geospatial data
    Datetime: It supplies classes for manipulating dates and times.
    Opencv (cv2): A powerful library for working with images in Python. Here, it is used for resampling rasters.
    OS: It provides functions for creating and removing a directory (folder).

In [2]:
import numpy as np
# import matplotlib.pyplot as plt
import rasterio
import gdal
from datetime import datetime
import os
import cv2
import xarray
# from multiprocessing import Pool, cpu_count

## Daily ET Estimation

In [3]:
import pyTSEB.TSEB as TSEB 

    The users can obtain detailed information about the input and outputs of pyTSEB.TSEB library by 
    calling the selected method, help(TSEB.TSEB_PT).

### Functions

***
<div align="justify"> In the following step, the input parameters are prepared using multiple functions. Most of these functions are from net_radiation (rad), meteo_utils (met), and resistances (res) packages. Moreover, two other functions are presented to estimate effective atmospheric emissivity and longwave atmospheric irradiance above the canopy:

</div>

***

In [4]:
def calc_emiss_atm(ea, t_a_k):
    '''Atmospheric emissivity

    Estimates the effective atmospheric emissivity for clear sky.

    Parameters
    ----------
    ea : float
        atmospheric vapour pressure (mb).
    t_a_k : float
        air temperature (Kelvin).

    Returns
    -------
    emiss_air : float
        effective atmospheric emissivity.

    References
    ----------
    .. [Brutsaert1975] Brutsaert, W. (1975) On a derivable formula for long-wave radiation
        from clear skies, Water Resour. Res., 11(5), 742-744,
        htpp://dx.doi.org/10.1029/WR011i005p00742.'''

    emiss_air = 1.24 * (ea / t_a_k)**(1. / 7.)  # Eq. 11 in [Brutsaert1975]_

    return np.asarray(emiss_air)

In [5]:
def calc_longwave_irradiance(ea, t_a_k, p, z_T, h_C):
    '''Longwave irradiance

    Estimates longwave atmospheric irradiance from clear sky.
    By default there is no lapse rate correction unless air temperature
    measurement height is considerably different than canopy height, (e.g. when
    using NWP gridded meteo data at blending height)

    Parameters
    ----------
    ea : float
        atmospheric vapour pressure (mb).
    t_a_k : float
        air temperature (K).
    p : float
        air pressure (mb)
    z_T: float
        air temperature measurement height (m), default 2 m.
    h_C: float
        canopy height (m), default 2 m,

    Returns
    -------
    L_dn : float
        Longwave atmospheric irradiance (W m-2) above the canopy
    '''

    lapse_rate = TSEB.met.calc_lapse_rate_moist(t_a_k, ea, p)
    t_a_surface = t_a_k - lapse_rate * (h_C - z_T)
    emisAtm = calc_emiss_atm(ea, t_a_surface)
    L_dn = emisAtm * TSEB.met.calc_stephan_boltzmann(t_a_surface)
    return np.asarray(L_dn)

***
<div align="justify"> The following two functions convert air and dew temperatures to the blending height, which is 100m. 

</div>

***

In [6]:
def calc_air_temperature_blending_height(ta, z_1, z_0):
    """
    # First move dry air adiabiatically
    ta_bh = ta - DRY_LAPSE_RATE * (z_1 - z_0)

    # Get cases when condensation occurs
    sat = np.logical_and.reduce((np.isfinite(ta_bh),
                                 np.isfinite(td),
                                 ta_bh < td))
    # Get condensation elevation of those cases
    z_sat = z_0[sat] - (td[sat] - ta[sat]) / DRY_LAPSE_RATE
    # Compute adiabatic lapse rate of moist air
    ea = calc_vapour_pressure(td[sat])
    p = calc_pressure_height(p[sat], ta[sat], ea, z_0[sat], z_sat)
    lapse_rate_moist = met.calc_lapse_rate_moist(td[sat].astype(np.float32),
                                                 ea.astype(np.float32),
                                                 p.astype(np.float32))
    # Move moist air adiabatically from elevation when condensation occurs
    # to destination height
    ta_bh[sat] = td[sat] - lapse_rate_moist * (z_1[sat] - z_sat)"""
    """
    ea = calc_vapour_pressure(td)
    lapse_rate_moist = met.calc_lapse_rate_moist(ta.astype(np.float32),
                                                 ea.astype(np.float32),
                                                 p.astype(np.float32))
    ta_bh = ta - lapse_rate_moist * (z_1 - z_0)
    """
    ta_bh = ta - ENVIRONMENTAL_LAPSE_RATE * (z_1 - z_0)
    return ta_bh

In [7]:
def calc_dew_temperature_blending_height(td, z_1, z_0):
    lapse_rate = TDEW_LAPSE_RATE
    td_bh = td - lapse_rate * (z_1 - z_0)
    return td_bh

***
<div align="justify"> Horizontal and Vertical Wind Speeds at 100 m height are acquired from ERA-5 and are used to calculate wind speed using the below function:

</div>

***

In [8]:
def calc_wind_speed(u, v):
    ws = (u ** 2 + v ** 2) ** 0.5
    ws = np.maximum(ws, 1.0)
    return ws

***
<div align="justify"> To obtain Sun zenith angle (sza), the below function is implemented using the coordinates of the target, central longitude of the time zone of the site, day of year, year, and time of LST acquisition.

</div>

***

In [9]:
def calc_sun_angles(lat, lon, stdlon, doy, ftime):
    """Calculates the Sun Zenith and Azimuth Angles (SZA & SAA).

    Parameters
    ----------
    lat : float
        latitude of the site (degrees).
    long : float
        longitude of the site (degrees).
    stdlng : float
        central longitude of the time zone of the site (degrees).
    doy : float
        day of year of measurement (1-366).
    ftime : float
        time of measurement (decimal hours).

    Returns
    -------
    sza : float
        Sun Zenith Angle (degrees).
    saa : float
        Sun Azimuth Angle (degrees).
    """

    lat, lon, stdlon, doy, ftime = map(
        np.asarray, (lat, lon, stdlon, doy, ftime))

    # Calculate declination
    declination = 0.409 * np.sin((2.0 * np.pi * doy / 365.0) - 1.39)
    EOT = (0.258 * np.cos(declination) - 7.416 * np.sin(declination)
           - 3.648 * np.cos(2.0 * declination) - 9.228 * np.sin(2.0 * declination))
    LC = (stdlon - lon) / 15.
    time_corr = (-EOT / 60.) + LC
    solar_time = ftime - time_corr

    # Get the hour angle
    w = np.asarray((solar_time - 12.0) * 15.)

    # Get solar elevation angle
    sin_thetha = (np.cos(np.radians(w)) * np.cos(declination) * np.cos(np.radians(lat))
                  + np.sin(declination) * np.sin(np.radians(lat)))
    sun_elev = np.arcsin(sin_thetha)

    # Get solar zenith angle
    sza = np.pi / 2.0 - sun_elev
    sza = np.asarray(np.degrees(sza))

    # Get solar azimuth angle
    cos_phi = np.asarray(
        (np.sin(declination) * np.cos(np.radians(lat))
         - np.cos(np.radians(w)) * np.cos(declination) * np.sin(np.radians(lat)))
        / np.cos(sun_elev))
    saa = np.zeros(sza.shape)
    saa[w <= 0.0] = np.degrees(np.arccos(cos_phi[w <= 0.0]))
    saa[w > 0.0] = 360. - np.degrees(np.arccos(cos_phi[w > 0.0]))

    return np.asarray(sza), np.asarray(saa)

***
<div align="justify"> Green Fraction is important to better estimate ET during the crop senescence phase.

</div>

***

In [10]:
def calc_fg_gutman(ndvi, ndvi_min, ndvi_max):
    num = ndvi - ndvi_min
    denum = ndvi_max - ndvi_min

    fg = num/denum
    fg[fg<0] = 0
    fg[fg>1] = 1

    return fg

***
<div align="justify"> Canopy height (hc) estimation is important to take into account the phenological dynamics

</div>

***

In [11]:
def hc_from_lai(lai, hc_max, lai_max, hc_min=0):
    """
    Estimates canopy height from LAI
    Parameters
    ----------
    lai : array
        Actual Leaf Area Index
    hc_max : float or array
        Maximum Canopy Height at lai_max (m)
    lai_max : float or array
        LAI at which maximum height is achieved
    hc_min : float or array, optional
        Canopy height (m) at LAI=0
    
    Returns
    -------
    h_c : array
        Canopy height (m)
    """
    h_c = hc_min + lai * (hc_max - hc_min) / lai_max
    h_c = np.clip(h_c, hc_min, hc_max)
    return h_c

***
<div align="justify"> The downscaled LST images and Sentinel-2 NDVI data are the inputs to estimate ET, along with SRTM DEM and ERA-5. DEM data can be acquired through USGS.gov  at 30-m spatial resolution. As it was explained in the algorithms document, we need to download seven meteorological parameters, including 100m u- and v-components of wind (m s-1), 2m dewpoint temperature (K), 2m temperature (K), Surface pressure (Pa), Surface solar radiation downward, clear sky (J m-2), and Geopotential (m2 s-2). These parameters can be downloaded via Climate Data Store  manual or Python API.</div>

***

***
<div align="justify"> Considering the target year, month, and day of estimation, the input_date variable should be assigned by this format: “YYYY-MM-DD”.</div>

***


The directories for 20m LST (lst_path), VI (vi_path), ERA5 (era5_dir), and 20-m ET estimation (output_path) tiff files should refer to corresponding folders. The following code reads all tiff files of the directories. As it’s commented, the pathes should be changed to the local directories.

***
<div align="justify"> Sensor viewing zenith angle (i.e., VZA) directly from the Sentinel-3 SLSTR product, called 'sat_zenith_tn'.

</div>

***

In [12]:
lst_path = r"/jupyter_work/project/S3_L2_2022/LST_20m_Monthly_G10"  # Change this to the path of your folder
vza_path = r"/jupyter_playgroundpython3.10_2.0/jupyter_work/project/VZA_2022/VZA_G10" 
vi_path = r"/jupyter_playgroundpython3.10_2.0/jupyter_work/project/Sentinel2_Kenya/2022/G10_Monthly"  # Change this to the path of your folder
LST_files = sorted([file for file in os.listdir(lst_path) if file.endswith(".tif")])
VI_files = sorted([file for file in os.listdir(vi_path) if file.endswith(".tif")])
VZA_files = sorted([file for file in os.listdir(vza_path) if file.endswith(".tif")])
era5_dir = r'/jupyter_work/project/ERA5/2022/G10/hourly/'

***
<div align="justify"> Leaf Area Index (LAI) is one of the main input of TSEB-PT function, and to calculate several other inputs. The LAI values are calculated by NDVI maps, where invalid values are neglected.

Furthermore, water vapor pressure (ea, using met.calc_vapor_pressure), pressure (p), longwave atmospheric irradiance above the canopy (L_dn), the zero-plane displacement height (using res.calc_d_0), and aerodynamic roughness length (using res.calc_z_0M) are calculated inside the loop. It should be noted that air temperature measurement height (z_T) and  windspeed measurement height (z_u) are 100 m. 

Net shortwave radiation is one of the main input of TSEB_PT, which is the net shortwave radiation for soil and canopy below a canopy using radiative transfer model. It is calculated by rad.calc_Sn_Campbell function, which needs several input parameters.

Surface solar radiation downward (ssrd_dwscaled) and Sun zenith angle (sza) are used in rad.calc_difuse_ratio function to estimate the fraction of diffuse shortwave radiation, including diffuse fraction in the visible and NIR regions and fraction of total visible and NIR radiations. These parameters and ssrd_dwscaled then calculate broadband incoming beam and diffuse shortwave radiations.

Later, reflectance values in visible and NIR domains of canopy and soil, and transmittance values in visible and NIR domains for canopy are provided from Sentinel-2 data. Noted that they should be prepared as the same size of input tiff files.

Finally, net shortwave radiations of soil and canopy (Sn_C and Sn_S) are estimated by rad.calc_Sn_Campbell function using 14 parameters, including the above mentioned parameters, parameter for the ellipsoidal Leaf Angle Distribution function (x_LAD, default =1), and the directional effective LAI to be used in the beam radiation (LAI_eff).

After preparing these parameters, the function calculates 17 outputs through several iterations, Where L_nC (Canopy net longwave radiation) is selected as the main output to calculate Actual Daily Evaportranspiration.

</div>

***

In [13]:
# Constants

year = 2022
DRY_LAPSE_RATE = 9.8e-3  # K m-1
ENVIRONMENTAL_LAPSE_RATE = 6.5e-3  # K m-1
# Dew temperature lapse rate
TDEW_LAPSE_RATE = 2e-3  # K m-1
SZA_THRESHOLD_CORRECTION = 75
z_bh = 100 # Blending Height
GRAVITY = 9.80665

In [ ]:
for v in range(0, len(LST_files)):
    
    # Extract the date from the first LST file  len(LST_files)
    first_lst_date_string = LST_files[v].split("_")[1].split(".")[0]  # Remove the file extension
    first_lst_date = datetime.strptime(first_lst_date_string, "%Y%m%d")
    first_lst_year_month = first_lst_date.strftime("%Y-%m")  # Get year-month format

    print('first_lst_date = ',first_lst_date)
    closest_vi_file = None

    for vi_file in VI_files:
        
        # Extract the date from the VI file
        vi_date_string = vi_file.split("_")[3].split(".")[0]  # Extract the date string
        vi_date = datetime.strptime(vi_date_string, "%Y-%m")  # Convert to datetime object in year-month format

        # Get the year and month of the VI file
        vi_year_month = vi_date.strftime("%Y-%m")

        # Check if the year and month match
        if vi_year_month == first_lst_year_month:
            closest_vi_file = vi_file  # Assign the VI file if a match is found

    if closest_vi_file:
        print(f"First LST file {LST_files[v]} - OK. Closest VI file: {closest_vi_file}.  Same month: {first_lst_year_month}.")
        
        LST_20m = rasterio.open(fr'{lst_path}/{LST_files[v]}')
        NDVI_20m = rasterio.open(fr'{vi_path}/{closest_vi_file}')

        VZA_km = rasterio.open(fr'{vza_path}/{VZA_files[v]}')
        print(f"VZA File {VZA_files[v]}")
        
        doy = first_lst_date.timetuple().tm_yday
        print('Day of the year (DOY) =', doy)
        day_of_month = first_lst_date.day 
        month_string = first_lst_date.strftime("%B")
        
        u_wind = gdal.Open(fr'{era5_dir}cropped_u100_hourly_mean_{year}.tif')
        v_wind = gdal.Open(fr'{era5_dir}cropped_v100_hourly_mean_{year}.tif')
        sp = gdal.Open(fr'{era5_dir}cropped_sp_hourly_mean_{year}.tif')
        ssrd = gdal.Open(fr'{era5_dir}cropped_ssrd_hourly_mean_{year}.tif')
        ssrd_mean = gdal.Open(fr'{era5_dir}cropped_ssrd_{year}.tif')
        dt2 = gdal.Open(fr'{era5_dir}cropped_2md_hourly_mean_{year}.tif')
        ta = gdal.Open(fr'{era5_dir}cropped_2mt_hourly_mean_{year}.tif')
        dem_srtm = gdal.Open(fr'{era5_dir}cropped_SRTM_G10.tif')
        gh = gdal.Open(fr'{era5_dir}cropped_GH_hourly_mean_{year}.tif')

        nc_latlon = os.path.join(fr'{era5_dir}cropped_v100_hourly_mean_{year}.nc')
        with xarray.open_dataset(nc_latlon) as ds:
            # Extract latitude and longitude from the dataset
            lat = ds['lat'].values
            lon = ds['lon'].values

        LST_data = LST_20m.read(1).astype(np.float32)
        LST_data = np.ma.masked_invalid(LST_data)
        LST2 = np.ma.masked_where(LST_data < 0, LST_data)

        VZA_data = VZA_km.read(1).astype(np.float32)
        VZA_data = np.ma.masked_invalid(VZA_data)
        VZA_deg = np.ma.masked_where(VZA_data < 0, VZA_data)        
        VZA_dwscaled = cv2.resize(VZA_deg, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)
        
        dom_input = doy
        
        NDVI_data = NDVI_20m.read(1)
        NDVI2 = np.ma.masked_invalid(NDVI_data)
        
        # LAI calculation
        LAI_np = 0.57*np.exp(2.33*NDVI2)
        LAI3 = np.ma.masked_where(LAI_np < 0, LAI_np)
        
        u_date = u_wind.GetRasterBand(dom_input)
        v_date = v_wind.GetRasterBand(dom_input) 
        u_array = u_date.ReadAsArray()
        v_array = v_date.ReadAsArray()
        wind_speed = calc_wind_speed(u_array, v_array)
        wind_speed_dwscaled = cv2.resize(wind_speed, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)
        
        sp_date = sp.GetRasterBand(dom_input) 
        sp_array = sp_date.ReadAsArray()
        sp_dwscaled = cv2.resize(sp_array/100, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)

        ssrd_m_date = ssrd_mean.GetRasterBand(dom_input)
        ssrd_m_array = ssrd_m_date.ReadAsArray()
        ssrd_m_dwscaled = cv2.resize(ssrd_m_array/3600, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)
        
        ssrd_date = ssrd.GetRasterBand(dom_input)
        ssrd_array = ssrd_date.ReadAsArray()
        ssrd_dwscaled = cv2.resize(ssrd_array/3600, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)

        dem_srtm = dem_srtm.ReadAsArray()
        dem_dwscaled = cv2.resize(dem_srtm, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)

        gh_date = gh.GetRasterBand(dom_input) 
        gh_array = gh_date.ReadAsArray()
        gh_dwscaled = cv2.resize(gh_array, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)
        z = gh_dwscaled/GRAVITY
        
        z_1_100 = dem_dwscaled + z_bh
        
        dt2_date = dt2.GetRasterBand(dom_input)
        dt2_array = dt2_date.ReadAsArray()
        dt2_dwscaled_0 = cv2.resize(dt2_array, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)
        dt2_dwscaled = calc_dew_temperature_blending_height(td = dt2_dwscaled_0, z_1 = z_1_100, z_0 = 2.0 + z)
        
        ta_date = ta.GetRasterBand(dom_input)
        ta_array = ta_date.ReadAsArray()
        td = dt2_dwscaled_0
        ta_dwscaled_0 = cv2.resize(ta_array, (LST2.shape[1], LST2.shape[0]), interpolation=cv2.INTER_NEAREST)
        ta_dwscaled = calc_air_temperature_blending_height(ta = ta_dwscaled_0, z_1 = z_1_100, z_0 = 2.0 + z)
        
        lons, lats = np.meshgrid(lon, lat)
        lons, lats = lons.astype(float), lats.astype(float)
        sza, ssa  = calc_sun_angles(lat = lats, lon = lons, stdlon = 40, doy = doy, ftime =10)
        
        hc = hc_from_lai(lai = LAI3, hc_max = 1.2, lai_max = np.nanmax(LAI_np), hc_min=0)
        
        ea = TSEB.met.calc_vapor_pressure(T_K=dt2_dwscaled)
        p = sp_dwscaled
        L_dn = calc_longwave_irradiance(ea = ea, t_a_k = ta_dwscaled, p = sp_dwscaled, z_T = 100, h_C = hc)
        d_0_0 = TSEB.res.calc_d_0(h_C=hc)
        z_0 = TSEB.res.calc_z_0M(h_C=hc)
        
        difvis, difnir, fvis, fnir = TSEB.rad.calc_difuse_ratio(S_dn = ssrd_dwscaled, sza = np.mean(sza))
        
        skyl = difvis * fvis + difnir * fnir
        S_dn_dir = ssrd_dwscaled * (1.0 - skyl)
        S_dn_dif = ssrd_dwscaled * skyl
        
        emis_C=0.98 # leaf emissivity
        emis_S=0.95 # soil emissivity
        
        # Leaf spectral properties:{rho_vis_C: visible reflectance, tau_vis_C: visible transmittance, rho_nir_C: NIR reflectance, tau_nir_C: NIR transmittance}
        rho_vis_C=np.full(LAI3.shape, 0.05, np.float32)
        tau_vis_C=np.full(LAI3.shape, 0.08, np.float32)
        rho_nir_C=np.full(LAI3.shape, 0.32, np.float32)
        tau_nir_C=np.full(LAI3.shape, 0.33, np.float32) 
        
        # Soil spectral properties:{rho_vis_S: visible reflectance, rho_nir_S: NIR reflectance}
        rho_vis_S=np.full(LAI3.shape, 0.07, np.float32)
        rho_nir_S=np.full(LAI3.shape, 0.25, np.float32)

        # F = local LAI
        F = LAI3 / 1
        # calculate clumping index
        Omega0 = TSEB.CI.calc_omega0_Kustas(LAI = LAI_np, f_C = 1, x_LAD=1)
        Omega = TSEB.CI.calc_omega_Kustas(Omega0, np.mean(sza))
        LAI_eff = F * Omega

        
        Sn_C, Sn_S = TSEB.rad.calc_Sn_Campbell(lai = LAI3, sza = np.mean(sza), S_dn_dir = S_dn_dir, S_dn_dif = S_dn_dif, fvis = fvis,
                                               fnir = fnir, rho_leaf_vis = rho_vis_C, tau_leaf_vis = tau_vis_C, rho_leaf_nir = rho_nir_C, 
                                               tau_leaf_nir = tau_nir_C, rsoilv = rho_vis_S, rsoiln = rho_nir_S, x_LAD=1, LAI_eff=LAI_eff)

        print('mean of Sn_S = ',np.mean(Sn_S))
        
        z_0M, d = TSEB.res.calc_roughness(LAI=np.nanmean(LAI3), h_C=hc, w_C=1, landcover=11, f_c=None)
        z_0M_array = np.full(LAI3.shape, z_0M)
        d_array = np.full(LAI3.shape, d)
        fg = calc_fg_gutman(ndvi = NDVI2, ndvi_min = np.nanmin(NDVI_data), ndvi_max = np.nanmax(NDVI_data))
        
        # Input Parameters to TSEB_PT
        #   ----------
        Tr_K = LST2
        # float. Radiometric composite temperature (Kelvin).
        vza = VZA_dwscaled
        # float. View Zenith Angle (degrees).
        T_A_K = ta_dwscaled
        # float. Air temperature (Kelvin).
        u =  wind_speed_dwscaled  ## ***ERA5 two para
        # float. Wind speed above the canopy (m s-1).
        ea = ea
        # float. Water vapour pressure above the canopy (mb).
        p = sp_dwscaled 
        # float. Atmospheric pressure (mb), use 1013 mb by default.
        Sn_C = Sn_C
        # float. Canopy net shortwave radiation (W m-2).
        Sn_S =  Sn_S
        # float. Soil net shortwave radiation (W m-2).
        L_dn = L_dn
        # float. Downwelling longwave radiation (W m-2).
        LAI = LAI3
        # float. Effective Leaf Area Index (m2 m-2).
        h_C = hc 
        # float. Canopy height (m).
        emis_C = 0.98
        # float. Leaf emissivity.
        emis_S = 0.95
        # float. Soil emissivity.
        z_0M = z_0M_array 
        # float. Aerodynamic surface roughness length for momentum transfer (m).
        d_0 = d_array 
        # float. Zero-plane displacement height (m).
        z_u = 100 
        # float. Height of measurement of windspeed (m).
        z_T = 100 
        # float. Height of measurement of air temperature (m).
        # leaf_width : 
        # float, optional average/effective leaf width (m).
        z0_soil = 0.01    #optional
        # float, bare soil aerodynamic roughness length (m).
        alpha_PT = 1.26  #optional
        #  float, optional. Priestley Taylor coeffient for canopy potential transpiration, use 1.26 by default.
        x_LAD = 1        #optional
        # float. Campbell 1990 leaf inclination distribution function chi parameter.
        f_c = 1          #optional
        #float. Fractional cover.
        f_g = fg        # optional
        # float. Fraction of vegetation that is green.
        w_C = 1       #optional
        # float. Canopy width to height ratio.
        
        print('all parameters are ready for the et estimation')
        
        output = TSEB.TSEB_PT(Tr_K, vza, T_A_K, u, ea, p, Sn_C, Sn_S, L_dn, LAI3, h_C, emis_C, emis_S, 
                              z_0M, d_0, z_u, z_T, resistance_form=None, calcG_params=None, const_L=None, 
                              kB=0.0, massman_profile=None, verbose=True)
        
        len(output)
        ld = output[6]/ssrd_dwscaled
        heat_latent_scaled = ssrd_m_dwscaled * ld
        et_daily_p = TSEB.met.flux_2_evaporation(heat_latent_scaled, t_k=T_A_K, time_domain=24)
        
        output_file = f'/jupyter_work/project/ET_results/ET_2022/actual_ET_20m_{first_lst_date_string}.tif'
        # arid_aoi: 
        # Get the metadata from the NDVI raster
        LST_metadata = LST_20m.meta
        
        # et_daily_p = np.ma.masked_where(et_daily < 0, et_daily)
        # et_daily_p = et_daily_p.filled(np.nan)  # Fill masked values with np.nan
        
        # Create the destination metadata
        dst_metadata = {
            'driver': 'GTiff',
            'dtype': et_daily_p.dtype,  # Use the dtype of the masked array
            'nodata': np.nan,  # Set nodata to np.nan
            'width': et_daily_p.shape[1],
            'height': et_daily_p.shape[0],
            'count': 1,  # Assuming a single-band GeoTIFF
            'crs': LST_metadata['crs'],
            'transform': LST_metadata['transform'],
            'compress': 'lzw'
        }
        
        # Save the array as a GeoTIFF
        with rasterio.open(output_file, 'w', **dst_metadata) as dst:
            dst.write(et_daily_p, 1) 
